In [ ]:
!pip install torch transformers


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "InstaDeepAI/nucleotide-transformer-500m-human-ref"
# model_name = "InstaDeepAI/nucleotide-transformer-v2-500m-human"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).eval().to("cuda")


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-500m-human-ref and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sequence = "ACGTGTCAGTACGTAGCTAGCTAGCGTAGCTAGCATGCTAG"
inputs = tokenizer(sequence, return_tensors="pt", padding=True).to("cuda")


In [ ]:
with torch.no_grad():
    outputs = model(**inputs)


In [ ]:
emb = outputs.last_hidden_state[0]
print("Embedding shape:", emb.shape)
print(emb[0][:10])  # first 10 dims of first token


Embedding shape: torch.Size([12, 1280])
tensor([ 0.0477,  0.1165,  0.2291,  0.4606, -0.8741, -0.0446, -0.2876,  0.4760,
        -0.0599,  1.2505], device='cuda:0')


In [ ]:
import numpy as np
np.save("/content/nt_embedding.npy", emb.cpu().numpy())


**From their Repo**


In [1]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-a0azp14j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-a0azp14j
  Resolved https://github.com/huggingface/transformers.git to commit bb65d2d953a512609a86727b6de64035717b1d45
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.9/514.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11353670 sha256=1daed768c0e5353cf6591e822362d68ba8cf480d983073567c578d0a6eb231af
  Stored in directory: /tmp/pip-ephem-wheel-cache-ijkushu5/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Foun

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Import the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")
model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")

# Choose the length to which the input sequences are padded. By default, the
# model max length is chosen, but feel free to decrease it as the time taken to
# obtain the embeddings increases significantly with it.
max_length = tokenizer.model_max_length

# Create a dummy dna sequence and tokenize it
sequences = ["ATTCCGATTCCGATTCCG", "ATTTCTCTCTCTCTCTGAGATCGATCGATCGAT"]
tokens_ids = tokenizer.batch_encode_plus(sequences, return_tensors="pt", padding="max_length", max_length = max_length)["input_ids"]

# Compute the embeddings
attention_mask = tokens_ids != tokenizer.pad_token_id #attn only for non-padding token
torch_outs = model(
    tokens_ids,
    attention_mask=attention_mask,
    encoder_attention_mask=attention_mask,
    output_hidden_states=True
)

# Compute sequences embeddings
embeddings = torch_outs['hidden_states'][-1].detach().numpy()
print(f"Embeddings shape: {embeddings.shape}")
print(f"Embeddings per token: {embeddings}")

# Add embed dimension axis
attention_mask = torch.unsqueeze(attention_mask, dim=-1)

# Compute mean embeddings per sequence --> each token has a vector embedding. So for a sequence, its embedding = mean of all embeddings of tokens
mean_sequence_embeddings = torch.sum(attention_mask*embeddings, axis=-2)/torch.sum(attention_mask, axis=1)
print(f"Mean sequence embeddings: {mean_sequence_embeddings}")
print(mean_sequence_embeddings.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Embeddings shape: (2, 1000, 1280)
Embeddings per token: [[[ 1.57492027e-01  1.13886923e-01  5.17816007e-01 ... -5.90867043e-01
   -2.21313059e-01  2.29983434e-01]
  [-7.45242655e-01  5.95985115e-01 -3.31419200e-01 ...  8.41172576e-01
    4.50770482e-02 -9.24153209e-01]
  [-9.06623662e-01  4.38094199e-01 -3.89714718e-01 ...  8.07051241e-01
   -6.04511909e-02 -1.04576373e+00]
  ...
  [-2.71285772e-02  4.97533739e-01 -1.07760929e-01 ... -1.93919912e-01
   -8.10948610e-02 -9.70781147e-01]
  [-2.71285772e-02  4.97533739e-01 -1.07760929e-01 ... -1.93919912e-01
   -8.10948610e-02 -9.70781147e-01]
  [-2.71285772e-02  4.97533739e-01 -1.07760929e-01 ... -1.93919912e-01
   -8.10948610e-02 -9.70781147e-01]]

 [[ 1.69805110e-01  6.60612583e-02  4.40444887e-01 ... -7.11013138e-01
   -1.65098812e-02  2.33012408e-01]
  [ 5.35150290e-01 -3.88687223e-01 -8.37923467e-01 ... -9.35686231e-02
    1.77557021e-02 -1.87366223e+00]
  [-2.00464010e-01  3.68334390e-02 -1.17718503e-01 ...  3.02496135e-01
   -2.814

/tmp/ipython-input-3111323719.py:35: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  mean_sequence_embeddings = torch.sum(attention_mask*embeddings, axis=-2)/torch.sum(attention_mask, axis=1)


In [3]:
print(max_length) # 1000 tokens so 1000*6 bases

1000
